In [ ]:
"""
Hybrid Quantum-Classical System for Renewable Energy Source Classification

This system uses Quantum Reservoir Computing (QRC) for feature extraction and
Quantum Support Vector Classifier (QSVC) for classification of renewable energy sources.

Technologies:
- PennyLane: For quantum circuits (QRC and QSVC)
- Scikit-learn: For classical SVC and metrics
- NumPy: For numerical operations
- Matplotlib: For visualization
- Kaggle Dataset: Real weather data

Author: Beginner-friendly implementation with detailed comments
"""

import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pennylane as qml
from pennylane import numpy as pnp
import kagglehub
import pandas as pd
import os

# =============================================================================
# PHASE 1: SETUP AND DATA LOADING
# =============================================================================

# Global constraint: Does the area have access to water resources?
HAS_RIVER_IN_AREA = True

def download_kaggle_dataset():
    """
    Downloads the weather dataset from Kaggle.

    Returns:
        str: Path to the downloaded dataset
    """
    print("Downloading Kaggle weather dataset...")
    path = kagglehub.dataset_download("muthuj7/weather-dataset")
    print(f"Dataset downloaded to: {path}")
    return path

def load_and_preprocess_data(dataset_path):
    """
    Loads and preprocesses the weather dataset.

    The function reads CSV files from the dataset, extracts relevant weather features,
    and prepares them for quantum machine learning processing.

    Args:
        dataset_path (str): Path to the dataset directory

    Returns:
        tuple: (features, labels) where features is shape (n_samples, 3) and labels is shape (n_samples,)
    """
    print("Loading and preprocessing weather data...")

    # Find CSV files in the dataset directory
    csv_files = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))

    if not csv_files:
        print("No CSV files found. Generating mock data instead.")
        return generate_mock_data(days=365)

    print(f"Found {len(csv_files)} CSV file(s)")

    # Load the first CSV file (you can modify this to combine multiple files)
    df = pd.read_csv(csv_files[0])
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")

    # Extract relevant features based on available columns
    # Common weather dataset columns: temperature, humidity, wind_speed, pressure, etc.

    features_dict = {
        'solar_irradiance': None,
        'wind_speed': None,
        'water_flow': None
    }

    # Map dataset columns to our required features
    # Solar irradiance approximation: use temperature, humidity, and cloud cover
    if 'Temperature_C' in df.columns or 'temperature' in df.columns:
        temp_col = 'Temperature_C' if 'Temperature_C' in df.columns else 'temperature'
        # Normalize temperature to 0-1 range (assuming -30 to 50 Celsius range)
        features_dict['solar_irradiance'] = (df[temp_col] + 30) / 80
    elif 'Temperature' in df.columns:
        features_dict['solar_irradiance'] = (df['Temperature'] + 30) / 80

    # Wind speed
    if 'Wind_Speed_kmh' in df.columns:
        # Normalize wind speed (0-100 km/h range)
        features_dict['wind_speed'] = df['Wind_Speed_kmh'] / 100
    elif 'wind_speed' in df.columns:
        features_dict['wind_speed'] = df['wind_speed'] / 100
    elif 'Wind Speed_km/h' in df.columns:
        features_dict['wind_speed'] = df['Wind Speed_km/h'] / 100

    # Water flow approximation: use humidity and precipitation
    if 'Humidity' in df.columns:
        # Normalize humidity (0-100% range)
        features_dict['water_flow'] = df['Humidity'] / 100
    elif 'humidity' in df.columns:
        features_dict['water_flow'] = df['humidity'] / 100

    # If any features are missing, generate synthetic data
    n_samples = len(df)
    for key, value in features_dict.items():
        if value is None:
            print(f"Warning: {key} not found in dataset, generating synthetic data")
            features_dict[key] = np.random.rand(n_samples)

    # Combine features into a single array
    features = np.column_stack([
        features_dict['solar_irradiance'],
        features_dict['wind_speed'],
        features_dict['water_flow']
    ])

    # Clip values to 0-1 range
    features = np.clip(features, 0, 1)

    # Limit to 365 days if more data is available
    if len(features) > 365:
        features = features[:365]

    # Generate labels based on the features
    labels = np.array([get_label(feat) for feat in features])

    print(f"Preprocessed data shape: {features.shape}")
    print(f"Unique labels: {np.unique(labels)}")

    return features, labels

def generate_mock_data(days=365):
    """
    Generates synthetic weather data for testing when real data is unavailable.

    This function creates realistic-looking weather patterns with seasonal variations
    for solar irradiance, wind speed, and water flow.

    Args:
        days (int): Number of days to generate data for

    Returns:
        tuple: (features, labels) where features is shape (days, 3) and labels is shape (days,)
    """
    print(f"Generating mock data for {days} days...")

    # Create time array for seasonal patterns
    t = np.linspace(0, 2 * np.pi, days)

    # Solar irradiance: Higher in summer, lower in winter
    # Base sine wave + random noise
    solar_irradiance = 0.5 + 0.3 * np.sin(t) + 0.2 * np.random.rand(days)

    # Wind speed: More variable, slight seasonal pattern
    # Inverse sine (higher in winter) + more noise
    wind_speed = 0.5 - 0.2 * np.sin(t) + 0.3 * np.random.rand(days)

    # Water flow: Related to precipitation patterns
    # Complex seasonal pattern + noise
    water_flow = 0.4 + 0.25 * np.sin(t + np.pi/4) + 0.35 * np.random.rand(days)

    # Clip all values to [0, 1] range
    solar_irradiance = np.clip(solar_irradiance, 0, 1)
    wind_speed = np.clip(wind_speed, 0, 1)
    water_flow = np.clip(water_flow, 0, 1)

    # Stack features into a single array
    features = np.column_stack([solar_irradiance, wind_speed, water_flow])

    # Generate labels based on the features
    labels = np.array([get_label(feat) for feat in features])

    return features, labels

def get_label(features):
    """
    Determines the most suitable renewable energy source based on weather features.

    Logic:
    - Solar: Best when solar irradiance is dominant
    - Wind: Best when wind speed is dominant
    - Hydro: Best when water flow is dominant AND HAS_RIVER_IN_AREA is True
    - Geothermal: Stable baseline option (can be added)
    - Mixed: When no single source is clearly dominant

    Args:
        features (array): [solar_irradiance, wind_speed, water_flow]

    Returns:
        str: Energy source label
    """
    solar_irr, wind_spd, water_flw = features

    # Define threshold for "dominant" feature
    DOMINANCE_THRESHOLD = 0.4

    # Check for dominant features
    max_feature = max(solar_irr, wind_spd, water_flw)

    # Solar is dominant
    if solar_irr == max_feature and solar_irr > DOMINANCE_THRESHOLD:
        return "Solar"

    # Wind is dominant
    elif wind_spd == max_feature and wind_spd > DOMINANCE_THRESHOLD:
        return "Wind"

    # Water/Hydro is dominant
    elif water_flw == max_feature and water_flw > DOMINANCE_THRESHOLD:
        if HAS_RIVER_IN_AREA:
            return "Hydro"
        else:
            # If no river, fall back to next best option
            if solar_irr > wind_spd:
                return "Solar"
            else:
                return "Wind"

    # No clear dominant feature - check for secondary options
    # Geothermal: Stable baseline (assume moderate conditions)
    elif solar_irr < 0.3 and wind_spd < 0.3 and water_flw < 0.3:
        return "Geothermal"

    # Mixed: Multiple good options available
    else:
        return "Mixed"

# =============================================================================
# PHASE 2: QUANTUM RESERVOIR COMPUTER (QRC)
# =============================================================================

# Define number of qubits for the reservoir
N_QUBITS = 5

# Create quantum device
# default.qubit is a simulator that runs on classical hardware
dev_qrc = qml.device('default.qubit', wires=N_QUBITS)

def encode_input(features, n_qubits):
    """
    Encodes classical input features into quantum circuit parameters.

    This function maps 3 weather features to n_qubits parameters by repeating
    and scaling the features. This is a simple encoding scheme suitable for
    beginners.

    Args:
        features (array): Weather features [solar, wind, water]
        n_qubits (int): Number of qubits in the quantum circuit

    Returns:
        array: Encoded parameters of length n_qubits
    """
    # Repeat features to match number of qubits
    # For example, if n_qubits=5 and features has 3 elements,
    # we cycle through features: [f0, f1, f2, f0, f1]
    encoded = np.zeros(n_qubits)
    for i in range(n_qubits):
        encoded[i] = features[i % len(features)]

    # Scale to quantum rotation angles (0 to 2π)
    encoded = encoded * 2 * np.pi

    return encoded

@qml.qnode(dev_qrc)
def qrc_circuit(params, weights):
    """
    Quantum Reservoir Circuit using StronglyEntanglingLayers.

    This circuit represents the "quantum dynamics" of the reservoir.
    It takes the current reservoir state and input features, applies
    quantum gates to create entanglement, and produces a new reservoir state.

    Args:
        params (array): Encoded input features (rotation angles)
        weights (array): Trainable weights for entangling layers

    Returns:
        list: Expectation values of PauliZ for each qubit (the new reservoir state)
    """
    # Step 1: Encode input features as rotation gates on each qubit
    # RY rotation represents the input signal
    for i in range(N_QUBITS):
        qml.RY(params[i], wires=i)

    # Step 2: Apply strongly entangling layers
    # This creates quantum entanglement between qubits, allowing the reservoir
    # to capture complex patterns in the data
    # StronglyEntanglingLayers applies rotations and CNOTs to create entanglement
    qml.StronglyEntanglingLayers(weights, wires=range(N_QUBITS))

    # Step 3: Measure the expectation values
    # PauliZ measurements give values between -1 and +1
    # These become the new reservoir state
    return [qml.expval(qml.PauliZ(i)) for i in range(N_QUBITS)]

def run_qrc(daily_features_list, n_layers=2):
    """
    Runs the Quantum Reservoir Computer on a sequence of daily weather data.

    This is the main QRC driver function. It processes each day's weather features
    sequentially, maintaining a reservoir state that captures temporal patterns.

    Args:
        daily_features_list (array): Array of shape (n_days, 3) with daily weather features
        n_layers (int): Number of entangling layers in the quantum circuit

    Returns:
        array: Reservoir states for each day, shape (n_days, N_QUBITS)
    """
    n_days = len(daily_features_list)

    # Initialize random weights for the entangling layers
    # Shape: (n_layers, n_qubits, 3) because StronglyEntanglingLayers uses 3 rotations per qubit
    weights = pnp.random.randn(n_layers, N_QUBITS, 3, requires_grad=False) * 0.1

    # Initialize reservoir state (start with zeros)
    reservoir_state = np.zeros(N_QUBITS)

    # Store all reservoir states
    reservoir_states = []

    print(f"Running QRC on {n_days} days of data...")

    # Process each day sequentially
    for day_idx, features in enumerate(daily_features_list):
        # Encode the day's weather features
        encoded_input = encode_input(features, N_QUBITS)

        # Combine encoded input with previous reservoir state
        # This allows the reservoir to maintain memory of past states
        combined_params = encoded_input + reservoir_state * 0.5

        # Run the quantum circuit to get new reservoir state
        new_state = qrc_circuit(combined_params, weights)

        # Convert to numpy array
        reservoir_state = np.array(new_state)

        # Store the state
        reservoir_states.append(reservoir_state.copy())

        # Progress indicator
        if (day_idx + 1) % 50 == 0:
            print(f"  Processed {day_idx + 1}/{n_days} days")

    return np.array(reservoir_states)

# =============================================================================
# PHASE 3: GRAMIAN ANGULAR FIELD TRANSFORMATION
# =============================================================================

def vector_to_image(vector):
    """
    Converts a 1D reservoir state vector to a 2D image using Gramian Angular Summation Field.

    The GASF transformation creates a 2D representation that captures temporal correlations
    within the time series data. This "image" can then be classified by the QSVC.

    Mathematical process:
    1. Rescale vector values to [-1, 1] range
    2. Calculate arccos to get angles (Polar Coordinate Transformation)
    3. Compute GASF matrix: GASF[i,j] = cos(angle[i] + angle[j])

    Args:
        vector (array): 1D reservoir state vector

    Returns:
        array: 2D GASF matrix (image representation)
    """
    # Step 1: Rescale to [-1, 1] range
    # Min-max normalization then scale to [-1, 1]
    min_val = np.min(vector)
    max_val = np.max(vector)

    if max_val - min_val < 1e-10:  # Avoid division by zero
        vector_scaled = np.zeros_like(vector)
    else:
        vector_scaled = 2 * (vector - min_val) / (max_val - min_val) - 1

    # Clip to ensure values are in [-1, 1] (for arccos stability)
    vector_scaled = np.clip(vector_scaled, -1, 1)

    # Step 2: Convert to angles (Polar Coordinate Transformation)
    # arccos maps [-1, 1] to [0, π]
    angles = np.arccos(vector_scaled)

    # Step 3: Compute GASF matrix
    # GASF[i,j] = cos(angle[i] + angle[j])
    # This captures the correlation between different parts of the time series
    n = len(angles)
    gasf = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            gasf[i, j] = np.cos(angles[i] + angles[j])

    return gasf

def visualize_gasf_image(gasf_image, title="GASF Image"):
    """
    Visualizes a GASF image using matplotlib.

    Args:
        gasf_image (array): 2D GASF matrix
        title (str): Title for the plot
    """
    plt.figure(figsize=(6, 6))
    plt.imshow(gasf_image, cmap='viridis', interpolation='nearest')
    plt.colorbar(label='GASF Value')
    plt.title(title)
    plt.xlabel('Time Index')
    plt.ylabel('Time Index')
    plt.tight_layout()
    plt.savefig(f'gasf_image_{title.replace(" ", "_")}.png')
    print(f"Saved GASF visualization as gasf_image_{title.replace(' ', '_')}.png")
    plt.close()

# =============================================================================
# PHASE 4: QUANTUM SUPPORT VECTOR CLASSIFIER (QSVC)
# =============================================================================

# Define quantum device for QSVC
N_QUBITS_QSVC = 4  # Using fewer qubits for the feature map to reduce computation
dev_qsvc = qml.device('default.qubit', wires=N_QUBITS_QSVC)

@qml.qnode(dev_qsvc)
def quantum_feature_map(features):
    """
    Quantum feature map circuit for the QSVC kernel.

    This circuit embeds classical data (flattened GASF images) into a quantum state.
    The quantum kernel then computes similarity between data points by measuring
    the overlap of their quantum states.

    We use AngleEmbedding to map features to rotation angles on qubits.

    Args:
        features (array): Flattened GASF image (must match or be truncated to N_QUBITS_QSVC)

    Returns:
        array: Quantum state (not used directly, but kernel computes overlap)
    """
    # Truncate or pad features to match number of qubits
    if len(features) > N_QUBITS_QSVC:
        features = features[:N_QUBITS_QSVC]
    elif len(features) < N_QUBITS_QSVC:
        features = np.pad(features, (0, N_QUBITS_QSVC - len(features)), mode='constant')

    # Angle embedding: each feature becomes a rotation angle
    qml.AngleEmbedding(features, wires=range(N_QUBITS_QSVC))

    # Add entangling layer to increase expressiveness
    for i in range(N_QUBITS_QSVC - 1):
        qml.CNOT(wires=[i, i + 1])

    # Return the statevector for kernel computation
    return qml.state()

def prepare_qsvc_data(reservoir_states, labels):
    """
    Prepares data for QSVC by converting reservoir states to GASF images and flattening.

    Args:
        reservoir_states (array): Output from QRC, shape (n_samples, N_QUBITS)
        labels (array): Energy source labels

    Returns:
        tuple: (X_flat, y_encoded) where X_flat is flattened images and y_encoded is integer labels
    """
    print("Preparing data for QSVC...")

    # Convert each reservoir state to a GASF image
    gasf_images = []
    for state in reservoir_states:
        gasf_img = vector_to_image(state)
        gasf_images.append(gasf_img)

    # Visualize one example GASF image
    if len(gasf_images) > 0:
        visualize_gasf_image(gasf_images[0], title="Example GASF Image Day 1")

    # Flatten images for SVC input
    # Each GASF image is (N_QUBITS x N_QUBITS), flatten to (N_QUBITS^2,)
    X_flat = np.array([img.flatten() for img in gasf_images])

    # Apply PCA or feature selection to reduce dimensionality for quantum circuit
    # Since we only have N_QUBITS_QSVC qubits, we need to reduce features
    from sklearn.decomposition import PCA

    n_features_target = min(N_QUBITS_QSVC, X_flat.shape[1])
    pca = PCA(n_components=n_features_target)
    X_reduced = pca.fit_transform(X_flat)

    print(f"Reduced feature dimensions from {X_flat.shape[1]} to {X_reduced.shape[1]}")

    # Encode labels as integers
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(labels)

    print(f"Label mapping: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

    return X_reduced, y_encoded, label_encoder, pca

def train_qsvc(X_train, X_test, y_train, y_test):
    """
    Trains and evaluates the Quantum Support Vector Classifier.

    This function:
    1. Computes quantum kernel matrices for training and test data
    2. Trains a classical SVC using the quantum kernel
    3. Evaluates the model on test data

    Args:
        X_train, X_test: Feature matrices
        y_train, y_test: Label vectors

    Returns:
        tuple: (trained_svc, kernel_train, predictions, probabilities)
    """
    print("Training QSVC...")
    print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")

    # Compute quantum kernel matrix for training data
    print("Computing training kernel matrix...")
    # Note: This is computationally expensive for large datasets
    # For demonstration, we'll use a subset if data is too large

    max_train_samples = 100  # Limit for computational efficiency
    if len(X_train) > max_train_samples:
        print(f"Limiting training data to {max_train_samples} samples for efficiency")
        indices = np.random.choice(len(X_train), max_train_samples, replace=False)
        X_train = X_train[indices]
        y_train = y_train[indices]

    # Compute kernel matrix using quantum feature map
    # This computes <phi(x_i)|phi(x_j)> for all pairs of training samples
    kernel_train = qml.kernels.kernel_matrix(X_train, X_train, quantum_feature_map)

    print(f"Training kernel shape: {kernel_train.shape}")

    # Train classical SVC with precomputed quantum kernel
    svc = SVC(kernel='precomputed', probability=True, random_state=42)
    svc.fit(kernel_train, y_train)

    print("Computing test kernel matrix...")
    # Compute kernel between test and training data
    kernel_test = qml.kernels.kernel_matrix(X_test, X_train, quantum_feature_map)

    # Make predictions
    predictions = svc.predict(kernel_test)
    probabilities = svc.predict_proba(kernel_test)

    print("\nClassification Report:")
    print(classification_report(y_test, predictions))

    return svc, kernel_train, predictions, probabilities, X_train

# =============================================================================
# PHASE 5: ANNUAL ANALYSIS AND RECOMMENDATION
# =============================================================================

def analyze_full_year(features, reservoir_states, svc, X_train, label_encoder, pca):
    """
    Analyzes a full year of data and generates energy source recommendations.

    This function:
    1. Converts all reservoir states to GASF images
    2. Computes quantum kernel against training data
    3. Gets daily probability scores for each energy source
    4. Analyzes trends and identifies peak seasons
    5. Generates final recommendations

    Args:
        features: Original weather features
        reservoir_states: QRC output for all days
        svc: Trained QSVC model
        X_train: Training data (for kernel computation)
        label_encoder: Label encoder for class names
        pca: PCA transformer for dimensionality reduction

    Returns:
        dict: Analysis results with recommendations
    """
    print("\n" + "="*60)
    print("FULL YEAR ANALYSIS")
    print("="*60)

    n_days = len(features)

    # Convert all reservoir states to GASF images and flatten
    print("Converting reservoir states to GASF images...")
    gasf_images = [vector_to_image(state) for state in reservoir_states]
    X_full_flat = np.array([img.flatten() for img in gasf_images])

    # Apply same PCA transformation as training data
    X_full_reduced = pca.transform(X_full_flat)

    # Compute kernel between full year data and training data
    print("Computing full year kernel matrix...")
    kernel_full = qml.kernels.kernel_matrix(X_full_reduced, X_train, quantum_feature_map)

    # Get daily probabilities for each energy source
    daily_probabilities = svc.predict_proba(kernel_full)
    daily_predictions = svc.predict(kernel_full)

    # Get class names
    class_names = label_encoder.classes_
    n_classes = len(class_names)

    print(f"\nAnalyzing {n_days} days across {n_classes} energy sources...")

    # Calculate total scores for each energy type
    total_scores = {}
    for i, class_name in enumerate(class_names):
        total_scores[class_name] = np.sum(daily_probabilities[:, i])

    print("\nTotal Annual Scores:")
    for energy_type, score in sorted(total_scores.items(), key=lambda x: x[1], reverse=True):
        print(f"  {energy_type}: {score:.2f}")

    # Find best timeframes (30-day windows) for each energy type
    window_size = 30
    best_windows = {}

    for i, class_name in enumerate(class_names):
        # Calculate rolling average for this energy type
        class_probs = daily_probabilities[:, i]
        best_avg = 0
        best_start = 0

        for start_day in range(n_days - window_size + 1):
            window_avg = np.mean(class_probs[start_day:start_day + window_size])
            if window_avg > best_avg:
                best_avg = window_avg
                best_start = start_day

        # Convert day to month approximation
        best_month = (best_start // 30) + 1
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        month_name = month_names[min(best_month - 1, 11)]

        best_windows[class_name] = {
            'start_day': best_start,
            'month': month_name,
            'avg_score': best_avg
        }

    print("\nPeak Seasons:")
    for energy_type, window_info in best_windows.items():
        print(f"  {energy_type}: {window_info['month']} (avg score: {window_info['avg_score']:.3f})")

    # Generate recommendations
    sorted_types = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)

    primary_type, primary_score = sorted_types[0]
    primary_month = best_windows[primary_type]['month']

    recommendation = {
        'primary_type': primary_type,
        'primary_score': primary_score,
        'primary_peak_month': primary_month,
        'secondary_type': None,
        'secondary_score': None,
        'secondary_peak_month': None,
        'all_scores': total_scores,
        'best_windows': best_windows,
        'daily_probabilities': daily_probabilities,
        'daily_predictions': daily_predictions
    }

    # Check for secondary recommendation (within 90% of top score)
    if len(sorted_types) > 1:
        secondary_type, secondary_score = sorted_types[1]
        if secondary_score >= 0.9 * primary_score:
            secondary_month = best_windows[secondary_type]['month']
            recommendation['secondary_type'] = secondary_type
            recommendation['secondary_score'] = secondary_score
            recommendation['secondary_peak_month'] = secondary_month

    return recommendation

def print_final_recommendation(recommendation):
    """
    Prints the final recommendation in a user-friendly format.

    Args:
        recommendation (dict): Analysis results from analyze_full_year
    """
    print("\n" + "="*60)
    print("RENEWABLE ENERGY RECOMMENDATION REPORT")
    print("="*60)

    print(f"\n🌟 PRIMARY RECOMMENDATION: {recommendation['primary_type']}")
    print(f"   Total Annual Score: {recommendation['primary_score']:.2f}")
    print(f"   Peak Season: {recommendation['primary_peak_month']}")

    if recommendation['secondary_type']:
        print(f"\n⚡ SECONDARY RECOMMENDATION: {recommendation['secondary_type']}")
        print(f"   Total Annual Score: {recommendation['secondary_score']:.2f}")
        print(f"   Peak Season: {recommendation['secondary_peak_month']}")
        print(f"\n   Note: Secondary option is within 90% of primary score,")
        print(f"         indicating strong viability as a complementary source.")

    print("\n" + "="*60)

def visualize_annual_trends(recommendation, label_encoder):
    """
    Creates visualizations of annual energy trends.

    Args:
        recommendation (dict): Analysis results
        label_encoder: Label encoder for class names
    """
    daily_probs = recommendation['daily_probabilities']
    n_days = len(daily_probs)
    class_names = label_encoder.classes_

    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

    # Plot 1: Daily probability trends
    days = np.arange(n_days)
    for i, class_name in enumerate(class_names):
        ax1.plot(days, daily_probs[:, i], label=class_name, linewidth=2)

    ax1.set_xlabel('Day of Year')
    ax1.set_ylabel('Probability Score')
    ax1.set_title('Daily Renewable Energy Source Probabilities')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Total scores bar chart
    scores = [recommendation['all_scores'][name] for name in class_names]
    colors = ['#FDB462', '#80B1D3', '#B3DE69', '#FCCDE5', '#8DD3C7']
    ax2.bar(class_names, scores, color=colors[:len(class_names)])
    ax2.set_ylabel('Total Annual Score')
    ax2.set_title('Total Annual Suitability by Energy Source')
    ax2.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig('annual_energy_analysis.png', dpi=150)
    print("\nSaved visualization as annual_energy_analysis.png")
    plt.close()

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """
    Main execution function that orchestrates the entire quantum ML pipeline.
    """
    print("="*60)
    print("HYBRID QUANTUM-CLASSICAL RENEWABLE ENERGY CLASSIFIER")
    print("="*60)
    print("\nThis system uses:")
    print("  • Quantum Reservoir Computing (QRC) for feature extraction")
    print("  • Gramian Angular Field (GAF) transformation")
    print("  • Quantum Support Vector Classifier (QSVC) for classification")
    print("="*60)

    # PHASE 1: Load data
    print("\n[PHASE 1] Loading data...")
    try:
        dataset_path = download_kaggle_dataset()
        features, labels = load_and_preprocess_data(dataset_path)
    except Exception as e:
        print(f"Error loading Kaggle dataset: {e}")
        print("Falling back to mock data generation...")
        features, labels = generate_mock_data(days=365)

    print(f"Loaded {len(features)} days of weather data")
    print(f"Features shape: {features.shape}")
    print(f"Unique labels: {np.unique(labels)}")

    # PHASE 2: Run Quantum Reservoir Computer
    print("\n[PHASE 2] Running Quantum Reservoir Computer...")
    reservoir_states = run_qrc(features, n_layers=2)
    print(f"Generated reservoir states shape: {reservoir_states.shape}")

    # PHASE 3: Convert to GASF images
    print("\n[PHASE 3] Converting to GASF images...")
    X_processed, y_encoded, label_encoder, pca = prepare_qsvc_data(reservoir_states, labels)

    # Apply constraint: Remove Hydro class if no river
    if not HAS_RIVER_IN_AREA:
        print("\nApplying constraint: HAS_RIVER_IN_AREA = False")
        print("Filtering out 'Hydro' samples...")
        hydro_label = None
        if 'Hydro' in label_encoder.classes_:
            hydro_label = label_encoder.transform(['Hydro'])[0]
            mask = y_encoded != hydro_label
            X_processed = X_processed[mask]
            y_encoded = y_encoded[mask]
            print(f"Removed {np.sum(~mask)} Hydro samples")
            print(f"Remaining samples: {len(X_processed)}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    # PHASE 4: Train QSVC
    print("\n[PHASE 4] Training Quantum Support Vector Classifier...")
    svc, kernel_train, predictions, probabilities, X_train_used = train_qsvc(
        X_train, X_test, y_train, y_test
    )

    # PHASE 5: Full year analysis
    print("\n[PHASE 5] Analyzing full year and generating recommendations...")
    recommendation = analyze_full_year(
        features, reservoir_states, svc, X_train_used, label_encoder, pca
    )

    # Print final recommendation
    print_final_recommendation(recommendation)

    # Create visualizations
    print("\nGenerating visualizations...")
    visualize_annual_trends(recommendation, label_encoder)

    print("\n" + "="*60)
    print("ANALYSIS COMPLETE")
    print("="*60)
    print("\nGenerated files:")
    print("  • gasf_image_Example_GASF_Image_Day_1.png")
    print("  • annual_energy_analysis.png")

    return recommendation

if __name__ == "__main__":
    # Set random seeds for reproducibility
    np.random.seed(42)

    # Run the complete pipeline
    recommendation = main()
